In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
sample_sub = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
numerical_cols = [cname for cname in train_df.columns if 
                train_df[cname].dtype in ['int64', 'float64']]

numerical_cols.remove('claim')
numerical_cols.remove('id')



In [ ]:
from sklearn.model_selection import train_test_split
X = train_df.drop(columns= ['id', 'claim'], axis=1)
y = train_df['claim']
X_test = test_df.drop(columns=['id']).copy()
# y_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size=0.2,random_state=42)

In [ ]:
# X_test

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Fill in the lines below: imputation
my_imputer = SimpleImputer(strategy='most_frequent') # Your code here
imputed_X_train =pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

# Imputation
my_imputer = SimpleImputer(strategy='most_frequent')
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))

final_X_test.columns = X_test.columns

final_X_train = imputed_X_train
final_X_valid = imputed_X_valid
final_X_test.head()

scaler = StandardScaler(with_mean=True)
final_X_train[numerical_cols] = scaler.fit_transform(final_X_train[numerical_cols])
final_X_valid[numerical_cols] = scaler.transform(final_X_valid[numerical_cols])
final_X_test[numerical_cols] = scaler.transform(final_X_test[numerical_cols])

In [ ]:
#Pipeline
catb_params = {
    'eval_metric' : 'AUC',
    'iterations': 15585, 
    'objective': 'CrossEntropy',
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.023575206684596582, 
    'reg_lambda': 36.30433203563295, 
    'random_strength': 43.75597655616195, 
    'depth': 7, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 1, 
    'subsample': 0.8227911142845009,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0
    }
    
model = CatBoostClassifier(**catb_params)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(final_X_train, y_train)


preds = clf.predict_proba(final_X_valid)[:,1]
fpr, tpr, _ = roc_curve(y_valid, preds)
score = auc(fpr, tpr)
print(score)

test_preds = clf.predict_proba(final_X_test)[:,1]

In [ ]:
#end
sample_sub['claim'] = test_preds
sample_sub.to_csv('PipelineCat.csv', index=False)